In [1]:
import pandas as pd

In [2]:
# import PGA data
pga_df = pd.read_csv('PGA_Data.csv')

In [10]:
# drop fantasy related and filler columns
pga_df = pga_df.drop(columns=['hole_DKP', 'hole_FDP', 'hole_SDP', 'streak_DKP',
       'streak_FDP', 'streak_SDP', 'n_rounds', 'made_cut', 'pos', 'finish_DKP',
       'finish_FDP', 'finish_SDP', 'total_DKP', 'total_FDP', 'total_SDP',
       'player', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',])

In [11]:
# make a column that is the finish, removing ties and turning 'CUT' to 0. 

# 0 could be changed to 99, NA, or whatever the cut line is (count in tournament / 2)

# note, the data column is an 'Object' type so it has strings and integers. 
# this methods turns everything to a string, makes its caclualtions, then turns to a float so we can do statistical calculations.

def adjust_finish(x):
    # this treats cut, dq, wd, etc. all as the same and assigns a value of 99
    # todo: determine if we want to treat these all the same
    # todo: because lower is better, NFs are treated as 99, but we can hone this in, 
    if x == 'CUT' or x == 'DQ' or x == 'WD' or x == 'MDF' or x == 'W/D':
        return '99'
    elif x.startswith('T'):
        return x[1:]
    else:
        return x
pga_df['finish_adj'] = pga_df['Finish'].astype(str).apply(adjust_finish).astype(float)


In [12]:
pga_df[pga_df['Player_initial_last'] == 'S. Scheffler'][[ 'Finish', 'finish_adj']]

,Finish,finish_adj
52,T3,3.0
159,CUT,99.0
648,T21,21.0
842,CUT,99.0
1114,T13,13.0
...,...,...
19977,T45,45.0
21884,T43,43.0
23699,CUT,99.0
26555,NaN,NaN


In [18]:
# look at a tournament and examine scoring stats with ranked leaderboad.

# initial findings after research: sg total is highly correlated with victory. but the best putter doesn't always win, for example.
# determining a model that weighs each of these stats to predict finish_adj is the early stages of preidction model, for each tournament.
pga_df[pga_df['tournament id'] ==  401353275][['Player_initial_last', 'sg_putt','sg_arg','sg_app',	'sg_ott','sg_t2g','sg_total','finish_adj']].sort_values('sg_putt', ascending= False)

,Player_initial_last,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total,finish_adj
9,C. Bezuidenhout,2.36,-0.11,-0.25,-0.76,-1.11,1.25,12.0
17,D. McCarthy,1.65,0.25,-0.90,-0.76,-1.40,0.25,28.0
2,A. Putnam,1.30,0.49,-0.27,-1.01,-0.80,0.50,23.0
60,T. Moore,1.28,0.02,-1.64,-0.17,-1.78,-0.50,44.0
40,M. Kuchar,1.27,-0.39,-0.61,-0.52,-1.52,-0.25,35.0
...,...,...,...,...,...,...,...,...
59,T. Merritt,-1.71,-0.29,0.59,-0.09,0.21,-1.50,58.0
63,W. Clark,-1.85,-0.61,-0.16,0.37,-0.40,-2.25,64.0
36,L. List,-2.14,-0.20,0.12,0.47,0.39,-1.75,61.0
49,S. Kim,-3.07,0.10,0.29,-0.58,-0.18,-3.25,67.0
